# VERSÃO DO AGENTE QUE USA TODAS AS TABELAS

In [0]:
%skip
# =========================================================
# DATAEXPERTS — CRIAÇÃO DO NL → SQL AGENT (GOLD - STAR)
# =========================================================

import mlflow
from mlflow.pyfunc import PythonModel
from mlflow.models.signature import ModelSignature
from mlflow.types.schema import Schema, ColSpec
from mlflow.deployments import get_deploy_client

# ---------------------------------------------------------
# CONFIGURAÇÕES
# ---------------------------------------------------------
EXPERIMENT_ID = "4074425516892918"
MODEL_NAME = "dataex_sql_agent"
LLM_ENDPOINT = "databricks-llama-4-maverick"

CATALOG = "dataexperts"
SCHEMA = "gold"
FACT_TABLE = "fato_vendas"

mlflow.set_experiment(experiment_id=EXPERIMENT_ID)

# ---------------------------------------------------------
# ASSINATURA DO MODELO
# ---------------------------------------------------------
input_schema = Schema([ColSpec("string", "question")])
output_schema = Schema([ColSpec("string", "sql"), ColSpec("string", "answer")])
signature = ModelSignature(inputs=input_schema, outputs=output_schema)
input_example = {"question": "Qual foi o faturamento total por mês em 2024?"}

# ---------------------------------------------------------
# CLIENTE LLM
# ---------------------------------------------------------
client = get_deploy_client("databricks")

# ---------------------------------------------------------
# AGENTE NL → SQL
# ---------------------------------------------------------
class DataExSQLAgent(PythonModel):

    @mlflow.trace
    def predict(self, context, model_input):
        question = model_input["question"]

        system_prompt = f"""
Você é um agente especialista em SQL Analytics no Databricks.

Modelo dimensional (Star Schema):

FATO:
- {CATALOG}.{SCHEMA}.{FACT_TABLE} (alias: f)

DIMENSÕES:
- dim_cliente (c)   f.sk_cliente = c.sk_cliente
- dim_produto (p)   f.sk_produto = p.sk_produto
- dim_localidade(l) f.sk_localidade = l.sk_localidade
- dim_data (d)      f.sk_data = d.sk_data

REGRAS:
- Use f como tabela principal
- Use apenas as tabelas acima
- Nunca invente colunas
- Para faturamento use SUM(f.valor_total)
- Para quantidade use SUM(f.quantidade)
- Retorne apenas SQL válido
"""

        user_prompt = f"""
Pergunta:
{question}

Retorne apenas o SQL.
"""

        response = client.predict(
            endpoint=LLM_ENDPOINT,
            inputs={
                "messages": [
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                "temperature": 0.0
            }
        )

        sql = response["choices"][0]["message"]["content"].strip()

        if context is None:
            return {"sql": sql, "answer": ""}

        df = spark.sql(sql)

        return {"sql": sql, "answer": df.toPandas().to_json(orient="records")}

# ---------------------------------------------------------
# LOG DO MODELO
# ---------------------------------------------------------
with mlflow.start_run(run_name="dataex-sql-agent-star"):
    mlflow.pyfunc.log_model(
        artifact_path="agent",
        python_model=DataExSQLAgent(),
        registered_model_name=MODEL_NAME,
        signature=signature,
        input_example=input_example
    )


In [0]:
# =========================================================
# DATAEXPERTS — USO DO NL → SQL AGENT (GOLD)
# =========================================================

import mlflow

MODEL_NAME = "dataex_sql_agent"

# Unity Catalog → usar alias criado para este agente
MODEL_URI = f"models:/{MODEL_NAME}@todas_as_tabelas"

# Carrega o agente
agent = mlflow.pyfunc.load_model(MODEL_URI)
print(f"✅ Agent carregado: {MODEL_URI}")

# Pergunta que queremos ao agente
pergunta = "Qual foi o faturamento por mês em 2024?"

resultado = agent.predict({"question": pergunta})

print("SQL GERADO (bruto pelo agente):")
print(resultado["sql"])

In [0]:
# =========================================================
# DATAEXPERTS — EXECUÇÃO AUTOMÁTICA DO SQL GERADO PELO AGENTE
# =========================================================

def limpar_sql(sql_texto: str) -> str:
    """
    Remove possíveis code fences ```sql ou ``` no SQL gerado pelo agente.
    """
    sql_texto = sql_texto.strip()
    if sql_texto.startswith("```sql"):
        sql_texto = sql_texto[len("```sql"):].strip()
    if sql_texto.startswith("```"):
        sql_texto = sql_texto[len("```"):].strip()
    if sql_texto.endswith("```"):
        sql_texto = sql_texto[:-3].strip()
    return sql_texto

# Limpa o SQL
sql_gerado = limpar_sql(resultado["sql"])

print("🚀 Executando SQL limpo pelo Spark:")
print(sql_gerado)

# Executa no Spark e exibe no Databricks
df_resultado = spark.sql(sql_gerado)
display(df_resultado)
